### Pipeline to get Prokaryote addendum genes

In [1]:
import requests
import pandas as pd
from functions import grouper, check_list_length
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep


In [2]:
def get_add_genes():
    response = requests.get('https://rest.kegg.jp/list/ag')
    lines = response.text.strip().split('\n')
    add_genes = ['ag:' + line.split("\t")[0] for line in lines]
    return add_genes
    

In [3]:
add_genes = get_add_genes()

Genes belonging to addendum are not included in the genomes that we are scrapping. The aim of this part is to add people with kb not included in the genomes annotated on kegg. The only problem here is that the lineage (which indicates the biological domain) may inadvertently not be indicated (causing an error in the function for the moment). To be updated, as well as the updating of the organisms made by KEGG,.


In [4]:
def get_prok_genes(add_genes):
    """
    Retrieve prokaryotic genes based on the LINEAGE information from the KEGG database.

    Parameters:
    - add_genes (list): List of gene identifiers.

    Returns:
    - list: List of prokaryotic genes.
    """
    prok_genes = {}
    domain = []
    pattern = re.compile(r'LINEAGE\s+(\w+);')
    pattern_extracts = re.compile(r'LINEAGE(.*?)ORGANISM', re.DOTALL)

    for i, group in enumerate(grouper(add_genes,10)):
        gene_group = [gene for gene in group if gene is not None]
        print(i)
        gene_group_url = '+'.join(gene_group)
        sleep(randint(10,20))
        response = requests.get(f'https://rest.kegg.jp/get/{gene_group_url}')
        
        match = re.findall(pattern, response.text)
        extracts = re.findall(pattern_extracts, response.text)
        
        indic_lin = [True if ";" in extract else False for extract in extracts]
        index_A = 0
        lin_value = []
        
        for element_B in indic_lin:
            if element_B and index_A < len(match):
                lin_value.append(match[index_A])
                index_A += 1
            else:
                lin_value.append('')
        
        prok_matches = {gene_group[k]:lin_value[k] for k in range(len(gene_group)) if lin_value[k] != 'Eukaryota'}
        prok_genes.update(prok_matches)

        
        
    return prok_genes

        
    

In [5]:
prok_matches = get_prok_genes(add_genes)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


KeyboardInterrupt: 

In [ ]:
set(prok_matches.values())

In [ ]:
domain = ['Archaea', 'Bacteria']

In [ ]:
prok_genes = [k for k, v in prok_matches.items() if v in domain]

In [ ]:
len(prok_genes)

In [15]:
def check_if_prot_list_API(page_response_text):

    pattern = re.compile(r'ORGANISM(.*?)NTSEQ', re.DOTALL)
    extracts = re.findall(pattern, page_response_text)
    indic_AA = [True if "AASEQ" in extract else False for extract in extracts]

    return indic_AA

def fetch_info_API(group):

    #regex
    pattern_KO = re.compile(r'ORTHOLOGY\s+(K\d{5})')
    pattern_seq = re.compile(r'NTSEQ\s+\d+(.*?)///', re.DOTALL)
    pattern_extract = re.compile(r'ENTRY(.*?)ORGANISM', re.DOTALL)

    #gene_ids
    gene_ids = [gene for gene in group if gene is not None]
    gene_group_url = '+'.join(gene_ids)
    gene_page_response = requests.get(f'https://rest.kegg.jp/get/{gene_group_url}')
    
    # Finding the KO
    match = re.findall(pattern_KO, gene_page_response.text)
    extracts = re.findall(pattern_extract, gene_page_response.text)
    indic_k0 = [True if "KO" in extract else False for extract in extracts]
    index_A = 0
    k0_value = []
    for element_B in indic_k0:
        if element_B and index_A < len(match):
            k0_value.append(match[index_A])
            index_A += 1
        else:
            k0_value.append('')
        
    #Finding the RNA NT sequence
    Nt_seq = re.findall(pattern_seq, gene_page_response.text)
    Nt_seq = [seq.replace(' ', '').replace('\n', '') for seq in Nt_seq]
    
    #Finding whether it is a protein or not
    prot = check_if_prot_list_API(gene_page_response.text)

    return gene_ids, k0_value, Nt_seq, prot


def scrape_add_genes(genes):
    dict_gene_org= []
    
    #we iterate over chunks of 10 genes_ids
    for i, group in enumerate(grouper(genes, 10)):
        
        results = {'gene_id': [], 'K0': [], 'Dna_SEQ': [], 'Prot': []}

        sleep(randint(40,100))
        gene_ids, k0_value, Nt_seq, prot = fetch_info_API(group)
              
        results['gene_id'].extend(gene_ids)
        results['K0'].extend(k0_value)
        results['Dna_SEQ'].extend(Nt_seq)
        results['Prot'].extend(prot)

        dict_gene_org.append(results)


        if i%10 == 0:
            print(i)
    df = pd.DataFrame(dict_gene_org)
    print(df)
    #Sometimes, due to high number of requests, requests get empty page, here we make sure that the dataframe is filled when written
    
    while ((df['K0'].apply(check_list_length)) | (df['Dna_SEQ'].apply(check_list_length)) | (df['Prot'].apply(check_list_length))).any():
        
        missing_info = df[df.apply(lambda row: any(check_list_length(cell) for cell in row[['K0', 'Dna_SEQ', 'Prot']]), axis=1)]
        print(len(missing_info))
        for index, row in missing_info.iterrows():
            gene_ids_list = row['gene_id']

            # Fetch information for missing gene IDs
            #sleep(randint(10,20))
            _, k0_values, dna_seq_values, prot_values = fetch_info_API(gene_ids_list)

            # Update the original DataFrame with the fetched information
            df.at[index, 'K0'] = k0_values
            df.at[index, 'Dna_SEQ'] = dna_seq_values
            df.at[index, 'Prot'] = prot_values

    df.to_csv('addendum_genes.csv' , index= False, mode = 'w')
    #upload_s3_csv(s3_f_folder, local_path_f_new + 'gene_' + organism +'_full.csv' )
    return results

In [ ]:
scrape_add_genes(prok_genes)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
                                               gene_id                    K0  \
0    [ag:AAA22071, ag:CAA11306, ag:CAB46491, ag:AAF...  [, , , , , , , , , ]   
1    [ag:CCK86740, ag:CAB51471, ag:AAC45086, ag:AFU...  [, , , , , , , , , ]   
2    [ag:AHM76779, ag:AHA80105, ag:AHA80106, ag:AHM...  [, , , , , , , , , ]   
3    [ag:ACJ42146, ag:CAK95249, ag:CAK95248, ag:CAK...  [, , , , , , , , , ]   
4    [ag:CAK95241, ag:CAK95240, ag:CAK95239, ag:CAK...  [, , , , , , , , , ]   
..                                                 ...                   ...   
323  [ag:BAA12093, ag:BAA07747, ag:BAA07748, ag:AAB...  [, , , , , , , , , ]   
324  [ag:AJR19448, ag:AAA21906, ag:CAA43999, ag:AAC...  [, , , , , , , , , ]   
325  [ag:CAA11866, ag:AAQ08379, ag:BAD66877, ag:AAB...  [, , , , , , , , , ]   
326  [ag:GAP38911, ag:GAP38373, ag:AAC25032, ag:BAD...  [, , , , , , , , , ]   

In [19]:
len(prok_matches)

3299

In [13]:
type(prok_matches)

dict

In [29]:
gene_group = add_genes[170:180]
gene_group_url = '+'.join(gene_group)
pattern = re.compile(r'LINEAGE\s+(\w+);')

response = requests.get(f'https://rest.kegg.jp/get/{gene_group_url}')



In [33]:
lin_value

['Bacteria',
 'Bacteria',
 'Bacteria',
 '',
 'Eukaryota',
 'Eukaryota',
 'Eukaryota',
 'Eukaryota',
 'Eukaryota',
 'Eukaryota']

In [27]:
lines = response.text.strip().split('\n')
lines

['ENTRY       AAF89160          CDS       T10000',
 'SYMBOL      BlaB-7',
 'NAME        (KEGG) beta-lactamase class B BlaB-7',
 'ORTHOLOGY   K24162  beta-lactamase class B BlaB [EC:3.5.2.6]',
 'TAXONOMY    TAX:238',
 '  LINEAGE   Bacteria; Bacteroidota; Flavobacteriia; Flavobacteriales; Weeksellaceae; Elizabethkingia',
 'ORGANISM    Elizabethkingia meningoseptica',
 'MOTIF       Pfam: Lactamase_B ODP',
 'DBLINKS     NCBI-ProteinID: AAF89160',
 '            GB: AF189304',
 '            UniProt: Q9KJA8',
 'AASEQ       248',
 '            MKGLKGLLVLALGFTGLQVFGQQNPDIKIEKLKDNLYVYTTYNTFKGTKYAANAVYMVTD',
 '            KGIVVIDSPWGEDKFKSFTDEIYKKHGKKVIMNIATHSHDDRAGGLEYFGKLGAKTYSTK',
 '            MTDSILAKENKPRAKYTFDNNKSFKVGKTEFQVYYPGKGHTADNVVVWFPKDKVLVGGCI',
 '            VKSGDSKDLGFIGEAYVNDWTQSIHNIQQKFPDVQYVVAGHDDWKDQTSIQHTLDLISEY',
 '            QQKQKASN',
 'REFERENCE   PMID:10858348',
 '  AUTHORS   Bellais S, Aubert D, Naas T, Nordmann P',
 '  TITLE     Molecular and biochemical heterogeneity